# TorchIP: Lennard-Jones potential
An examples notebook that shows how to reconstruct a Lennard-Jones potential. 


TODO
- use trainer
- random sampling, test-train-split
- predict energy
- predict force

## Imports

In [ ]:
import sys
sys.path.append('../')

import torchip as tp
from torchip.config import CFG
from torchip.structure import Structure
from torchip.loaders import RunnerStructureLoader, read_structures
from torchip.utils import gradient
from torchip.descriptors import ASF, CutoffFunction, G2
from torchip.descriptors import DescriptorScaler
from torchip.potentials import NeuralNetworkPotential

import torch
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from scipy.optimize import curve_fit
import matplotlib.pylab as plt

np.random.seed(2022)
torch.manual_seed(2022);

# print(tp.__doc__)
# print(f"Import TorchIP {tp.__version__}")

In [ ]:
# Set device eigher cpu or cuda (gpu)
CFG.set("device", "cpu")

## Structure

In [ ]:
# Create structure loader
workdir = Path('/home/hossein/n2p2/examples/nnp-train/LJ')
sloader = RunnerStructureLoader(Path(workdir, "input.data")) 

In [ ]:
# # Read structures using get_data() method (including comments)
# for data in sloader.get_data():
#     structure = Structure(data, requires_grad=True)
#     break
# print('comment:', data['comment'])
# print(structure)
# structure.position

### Energy and force

In [ ]:
def potential(r, esp=1.0, sig=1.0):
    """
    Explicit definition of the Lennard-Jones potential (for debugging). 
    """
    tmp = (sig/r)**6
    return 4.0 * esp * (tmp**2 - tmp) 

def force(R, esp=1.0, sig=1.0):
    """
    Explicit definition of the Lennard-Jones force vector (for debugging).
    """
    R = torch.atleast_2d(R)
    r = torch.norm(R, dim=1)
    tmp = (sig/r)**6
    return 24.0 * esp / r**2 * (2.0*tmp**2 - tmp) * R

In [ ]:
data = defaultdict(list)

def get_val(x: torch.Tensor) -> np.ndarray:
    return x.detach().cpu().numpy() 

for structure in sloader.get_structure():
    r = structure.calculate_distance(aid=0, neighbors=1) 
    E = structure.total_energy #potential(r)
    F = structure.force #-gradient(E, structure.position)
    R = structure.position
#     print("r:", get_val(r))
#     print("E:", get_val(E))
#     print("F:", get_val(F))
#     print("R:", get_val(R))
#     print()
#     break

#     Rij = R[0] - R[1]
#     print("auto-diff:", get_val(F[0]))
#     print("trueforce:", get_val(force(Rij)[0]))
#     print()

    data['energy'].append(E.detach().cpu().numpy())
    data['distance'].append(r.detach().cpu().numpy())

In [ ]:
plt.scatter(data['distance'], data['energy'])
plt.xlabel("distance"); plt.ylabel("energy");

### Finding epsilon and sigma parameters

In [ ]:
x = np.array(data['distance'])[:, 0]
y = np.array(data['energy'])[:, 0]

popt, pcov = curve_fit(potential, x, y)
print(f"eps = {popt[0]}, sig = {popt[1]}")

## Potential

In [ ]:
pot = NeuralNetworkPotential(Path(workdir, "input.nn"))

### Descriptors

In [ ]:
# # Cutoff function
# r_cutoff, cutoff_type = 3.0, "tanh"
# cfn = CutoffFunction(r_cutoff, cutoff_type)

# # Descriptor
# asf = ASF("Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.00), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.25), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.50), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=0.75), "Ne")
# asf.register(G2(cfn, eta=1.0, r_shift=1.00), "Ne")

In [ ]:
pot.fit_scaler(sloader)
pot.load_scaler()

### Model

#### Training

In [ ]:
%time history = pot.fit_model(sloader)
pot.load_model()

In [ ]:
df = pd.DataFrame(history)
df[-30:].plot();

#### Energy

In [ ]:
structures = read_structures(sloader)
sid = 1

In [ ]:
def mse(predictions, targets):
    return ((predictions - targets) ** 2).mean()

def rmse(predictions, targets):
    return np.sqrt(mse(predictions, targets))

energy = pot(structures[sid])
E_pred = get_val(energy)
E_true = get_val(structures[sid].total_energy)
print("Predicted energy:\n", E_pred)
print("True energy:\n", E_true)
print("MSE:\n", mse(E_pred, E_true))
print("RMSE:\n", rmse(E_pred, E_true))

#### Force

In [ ]:
force = -gradient(energy, structures[sid].position)
F_pred = get_val(force)
F_true = get_val(structures[sid].force)
print("Predicted force: \n", F_pred )
print("True force:\n", F_true)
print("MSE:\n", mse(F_pred, F_true))
print("RMSE:\n", rmse(F_pred, F_true))